# When separating the adopted user group, users with the creation type Guest_Invite are noticeably higher (23% of the adopted user population compared to 18% for the entire population). This implies that we should encourage our users to invite friends. In addition to this analysis, linear regression could be used to review how each variable is correlated to the adopted user target. I would like to talk with you to gain more practical insight into the business and I would be happy to build this model with you if hired.

In [35]:
import pandas as pd
import numpy as np
file ='/Users/ryanpurcell/Desktop/relax_challenge/takehome_user_engagement.csv'
df = pd.read_csv(file)
df.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [36]:
df['time_stamp2'] = pd.to_datetime(df['time_stamp'])

In [37]:
def keep_repeat_users(df, visited=3):
	"""
	INPUT
	df: full dataframe of user activity
	visited = 3: minimum amount of times logged to be considered an adopted user
	OUTPUT
	new_df: dataframe containing all users who have logged on at least the 'visited' time, grouped by user_id
	"""
	import pandas as pd
	from pandas import DataFrame

	new_df = df.groupby('user_id').filter(lambda x: len(x) >= visited)
	return new_df

In [38]:
# Only keep users who have logged at least 3 days
repeated_users_df = keep_repeat_users(df)

In [39]:
grouped_users_df = repeated_users_df.groupby('user_id')

In [40]:
unique_active_peeps_df = keep_active_users(grouped_users_df)

In [41]:
unique_active_peeps_df['adopted_user'] = 1

In [42]:
adopted_user_info = pd.merge(unique_active_peeps_df, df1, how='outer',
                  left_on='user_id', right_on='object_id')

In [43]:
adopted_user_info['adopted_user'] = adopted_user_info['adopted_user'].fillna(0)

In [44]:
def keep_active_users(df):
	"""
	OUTPUT
	unique_active_peeps: a dataframe of unique adopted users
	"""
	import pandas as pd
	from pandas import DataFrame, Series

	active_peeps = df.filter(lambda x: active_users(period=7, days_logged=3, user=x) ==True)

	unique_active_peeps = DataFrame(Series.unique(active_peeps['user_id']))
	unique_active_peeps.columns = ['user_id']

	return unique_active_peeps

In [45]:

def active_users(period, days_logged, user):
	"""
	INPUT
	period: time period we want to look at, default 7
	days_logged: is the number of days of the period we want the user to have logged in, default 3
	user is the unique users
	OUPUT
	active_user: returns whether or not the user had 3 consecutive logins within a 7 day period
	"""
	import pandas as pd
	from pandas import DataFrame, Series

	visited = len(user.index) #get the number of times the user logged in
	i, count = 0, 1
	active_user = False

	while count < days_logged:
		if (i+2) < visited: #needs to be at least 3 entries left	
			if (user['time_stamp2'].iloc[i + 1] - user['time_stamp2'].iloc[i]) <= pd.Timedelta(days=period) and (user['time_stamp2'].iloc[i + 1] - user['time_stamp2'].iloc[i]) > pd.Timedelta(days=1) :
				count += 1 #logged in twice within a 7 day period
				new_timeframe = (user['time_stamp2'].iloc[i + 1] - user['time_stamp2'].iloc[i])
				if (user['time_stamp2'].iloc[i + 2] - user['time_stamp2'].iloc[i + 1]) <= new_timeframe and (user['time_stamp2'].iloc[i + 2] - user['time_stamp2'].iloc[i + 1]) > pd.Timedelta(days=1):
					active_user = True #they logged in three times within a 7 period window
					count += 1
				else: 
					i += 1
					count = 1
			else:
				i += 1
				count = 1
		else:
			count = days_logged
	return active_user

In [46]:
import pandas as pd
import numpy as np
file ='/Users/ryanpurcell/Desktop/relax_challenge/takehome_users.csv'
df1 = pd.read_csv(file, encoding = "ISO-8859-1")
df1.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [53]:
adopted_user_info

,user_id,adopted_user,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,2.0,1.0,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
1,10.0,1.0,10,2013-01-16 22:08:03,Santos Carla,CarlaFerreiraSantos@gustr.com,ORG_INVITE,1.401833e+09,1,1,318,4143.0
2,33.0,1.0,33,2014-03-11 06:29:09,Araujo José,JoseMartinsAraujo@cuvox.de,GUEST_INVITE,1.401518e+09,0,0,401,79.0
3,42.0,1.0,42,2012-11-11 19:05:07,Pinto Giovanna,GiovannaCunhaPinto@cuvox.de,SIGNUP,1.401045e+09,1,0,235,NaN
4,50.0,1.0,50,2012-06-03 11:02:08,Oliveira Gabrielly,GabriellyAlmeidaOliveira@gmail.com,GUEST_INVITE,1.350990e+09,0,0,61,50.0
5,53.0,1.0,53,2013-02-09 23:47:15,Ribeiro Camila,CamilaGomesRibeiro@gmail.com,GUEST_INVITE,1.367798e+09,0,0,37,3641.0
6,60.0,1.0,60,2014-03-21 22:56:03,McEacharn Archie,ArchieMcEacharn@gmail.com,ORG_INVITE,1.400195e+09,0,0,88,3463.0
7,63.0,1.0,63,2012-12-20 16:30:52,Martins Leonor,LeonorAraujoMartins@gmail.com,SIGNUP_GOOGLE_AUTH,1.401899e+09,0,0,203,NaN
8,69.0,1.0,69,2012-06-29 10:15:18,Bowen Alisha,AlishaBowen@gustr.com,GUEST_INVITE,1.401445e+09,0,0,107,5022.0
9,74.0,1.0,74,2013-01-14 00:02:41,Farrell Jack,JackFarrell@gmail.com,SIGNUP_GOOGLE_AUTH,1.377303e+09,0,0,89,NaN


In [58]:
afilter = adopted_user_info['adopted_user']==1.0

In [59]:
adopted_user_info[afilter]

,user_id,adopted_user,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,2.0,1.0,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
1,10.0,1.0,10,2013-01-16 22:08:03,Santos Carla,CarlaFerreiraSantos@gustr.com,ORG_INVITE,1.401833e+09,1,1,318,4143.0
2,33.0,1.0,33,2014-03-11 06:29:09,Araujo José,JoseMartinsAraujo@cuvox.de,GUEST_INVITE,1.401518e+09,0,0,401,79.0
3,42.0,1.0,42,2012-11-11 19:05:07,Pinto Giovanna,GiovannaCunhaPinto@cuvox.de,SIGNUP,1.401045e+09,1,0,235,NaN
4,50.0,1.0,50,2012-06-03 11:02:08,Oliveira Gabrielly,GabriellyAlmeidaOliveira@gmail.com,GUEST_INVITE,1.350990e+09,0,0,61,50.0
5,53.0,1.0,53,2013-02-09 23:47:15,Ribeiro Camila,CamilaGomesRibeiro@gmail.com,GUEST_INVITE,1.367798e+09,0,0,37,3641.0
6,60.0,1.0,60,2014-03-21 22:56:03,McEacharn Archie,ArchieMcEacharn@gmail.com,ORG_INVITE,1.400195e+09,0,0,88,3463.0
7,63.0,1.0,63,2012-12-20 16:30:52,Martins Leonor,LeonorAraujoMartins@gmail.com,SIGNUP_GOOGLE_AUTH,1.401899e+09,0,0,203,NaN
8,69.0,1.0,69,2012-06-29 10:15:18,Bowen Alisha,AlishaBowen@gustr.com,GUEST_INVITE,1.401445e+09,0,0,107,5022.0
9,74.0,1.0,74,2013-01-14 00:02:41,Farrell Jack,JackFarrell@gmail.com,SIGNUP_GOOGLE_AUTH,1.377303e+09,0,0,89,NaN


In [65]:
adopted_user_info[afilter].groupby('creation_source').count()/(adopted_user_info[afilter].groupby('creation_source').count()).sum()

,user_id,adopted_user,object_id,creation_time,name,email,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
creation_source,,,,,,,,,,,
GUEST_INVITE,0.226006,0.226006,0.226006,0.226006,0.226006,0.226006,0.226006,0.226006,0.226006,0.226006,0.39545
ORG_INVITE,0.345511,0.345511,0.345511,0.345511,0.345511,0.345511,0.345511,0.345511,0.345511,0.345511,0.60455
PERSONAL_PROJECTS,0.102786,0.102786,0.102786,0.102786,0.102786,0.102786,0.102786,0.102786,0.102786,0.102786,0.00000
SIGNUP,0.180186,0.180186,0.180186,0.180186,0.180186,0.180186,0.180186,0.180186,0.180186,0.180186,0.00000
SIGNUP_GOOGLE_AUTH,0.145511,0.145511,0.145511,0.145511,0.145511,0.145511,0.145511,0.145511,0.145511,0.145511,0.00000


In [64]:
adopted_user_info.groupby('creation_source').count()/(adopted_user_info.groupby('creation_source').count()).sum()

,user_id,adopted_user,object_id,creation_time,name,email,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
creation_source,,,,,,,,,,,
GUEST_INVITE,0.226006,0.180250,0.180250,0.180250,0.180250,0.180250,0.179984,0.180250,0.180250,0.180250,0.337073
ORG_INVITE,0.345511,0.354500,0.354500,0.354500,0.354500,0.354500,0.361328,0.354500,0.354500,0.354500,0.662927
PERSONAL_PROJECTS,0.102786,0.175917,0.175917,0.175917,0.175917,0.175917,0.086592,0.175917,0.175917,0.175917,0.000000
SIGNUP,0.180186,0.173917,0.173917,0.173917,0.173917,0.173917,0.215120,0.173917,0.173917,0.173917,0.000000
SIGNUP_GOOGLE_AUTH,0.145511,0.115417,0.115417,0.115417,0.115417,0.115417,0.156976,0.115417,0.115417,0.115417,0.000000


In [69]:
adopted_user_info[nafilter].groupby('creation_source').count()/(adopted_user_info[nafilter].groupby('creation_source').count()).sum()

,user_id,adopted_user,object_id,creation_time,name,email,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
creation_source,,,,,,,,,,,
GUEST_INVITE,NaN,0.173134,0.173134,0.173134,0.173134,0.173134,0.169673,0.173134,0.173134,0.173134,0.327266
ORG_INVITE,NaN,0.355898,0.355898,0.355898,0.355898,0.355898,0.364872,0.355898,0.355898,0.355898,0.672734
PERSONAL_PROJECTS,NaN,0.187289,0.187289,0.187289,0.187289,0.187289,0.082963,0.187289,0.187289,0.187289,0.000000
SIGNUP,NaN,0.172942,0.172942,0.172942,0.172942,0.172942,0.222947,0.172942,0.172942,0.172942,0.000000
SIGNUP_GOOGLE_AUTH,NaN,0.110737,0.110737,0.110737,0.110737,0.110737,0.159545,0.110737,0.110737,0.110737,0.000000


In [60]:
adopted_user_info

,user_id,adopted_user,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,2.0,1.0,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
1,10.0,1.0,10,2013-01-16 22:08:03,Santos Carla,CarlaFerreiraSantos@gustr.com,ORG_INVITE,1.401833e+09,1,1,318,4143.0
2,33.0,1.0,33,2014-03-11 06:29:09,Araujo José,JoseMartinsAraujo@cuvox.de,GUEST_INVITE,1.401518e+09,0,0,401,79.0
3,42.0,1.0,42,2012-11-11 19:05:07,Pinto Giovanna,GiovannaCunhaPinto@cuvox.de,SIGNUP,1.401045e+09,1,0,235,NaN
4,50.0,1.0,50,2012-06-03 11:02:08,Oliveira Gabrielly,GabriellyAlmeidaOliveira@gmail.com,GUEST_INVITE,1.350990e+09,0,0,61,50.0
5,53.0,1.0,53,2013-02-09 23:47:15,Ribeiro Camila,CamilaGomesRibeiro@gmail.com,GUEST_INVITE,1.367798e+09,0,0,37,3641.0
6,60.0,1.0,60,2014-03-21 22:56:03,McEacharn Archie,ArchieMcEacharn@gmail.com,ORG_INVITE,1.400195e+09,0,0,88,3463.0
7,63.0,1.0,63,2012-12-20 16:30:52,Martins Leonor,LeonorAraujoMartins@gmail.com,SIGNUP_GOOGLE_AUTH,1.401899e+09,0,0,203,NaN
8,69.0,1.0,69,2012-06-29 10:15:18,Bowen Alisha,AlishaBowen@gustr.com,GUEST_INVITE,1.401445e+09,0,0,107,5022.0
9,74.0,1.0,74,2013-01-14 00:02:41,Farrell Jack,JackFarrell@gmail.com,SIGNUP_GOOGLE_AUTH,1.377303e+09,0,0,89,NaN


In [67]:
nafilter = adopted_user_info['adopted_user']==0.0

In [68]:
adopted_user_info[nafilter]

,user_id,adopted_user,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
1615,NaN,0.0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1616,NaN,0.0,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
1617,NaN,0.0,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
1618,NaN,0.0,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0
1619,NaN,0.0,6,2013-12-17 03:37:06,Cunha Eduardo,EduardoPereiraCunha@yahoo.com,GUEST_INVITE,1.387424e+09,0,0,197,11241.0
1620,NaN,0.0,7,2012-12-16 13:24:32,Sewell Tyler,TylerSewell@jourrapide.com,SIGNUP,1.356010e+09,0,1,37,NaN
1621,NaN,0.0,8,2013-07-31 05:34:02,Hamilton Danielle,DanielleHamilton@yahoo.com,PERSONAL_PROJECTS,NaN,1,1,74,NaN
1622,NaN,0.0,9,2013-11-05 04:04:24,Amsel Paul,PaulAmsel@hotmail.com,PERSONAL_PROJECTS,NaN,0,0,302,NaN
1623,NaN,0.0,11,2013-12-26 03:55:54,Paulsen Malthe,MaltheAPaulsen@gustr.com,SIGNUP,1.388117e+09,0,0,69,NaN
1624,NaN,0.0,12,2014-04-17 23:48:38,Mathiesen Lærke,LaerkeLMathiesen@cuvox.de,ORG_INVITE,NaN,0,0,130,9270.0
